#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

## Insert your ID number ("numero di matricola") below

In [1]:
#put here your ``numero di matricola''
numero_di_matricola = #RANDOM VALUE

In [2]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [3]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [4]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $m_{train}$ samples, the one for choosing among different models will consist of $m_{val}$ sampels, while the other part consists of $m_{test}=m - m_{traing} - m_{val}$ samples ($m$ is the total number of samples in the data).

In [5]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [6]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [7]:
#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model
mlp = MLPRegressor( hidden_layer_sizes=(1,), solver ='lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
mlp.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:")
print(str(1 - mlp.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:")
print(str(1 - mlp.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp.intercepts_)

1 - coefficient of determination on training data:
0.26394813650265647
1 - coefficient of determination on validation data:
0.30404986767777964
[array([[-214.32073761],
       [ 268.55443633],
       [ 523.02509525],
       [ -60.45970196],
       [   3.99824051],
       [ 709.0869042 ],
       [ 294.01089124],
       [ 136.49888896],
       [ 813.47952614],
       [ 492.65027485],
       [ 163.05923478],
       [-580.53859833],
       [  37.99224465],
       [-203.08392377],
       [ 598.76480958],
       [-141.78113496],
       [ 146.56038073],
       [ -27.02298029]]), array([[141.45513977]])]
[array([3785.55073619]), array([-38.89073385])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [8]:
from sklearn import linear_model

LR = linear_model.LinearRegression()

LR.fit(Xtrain_scaled, Ytrain)

print("1 - coefficient of determination on training data:")
print(str(1 - LR.score(Xtrain_scaled,Ytrain)))

print("1 - coefficient of determination on validation data:")
print(str(1 - LR.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(LR.coef_)

#let's print the coefficient for the bias (i.e., the bias)
print(LR.intercept_)

1 - coefficient of determination on training data:
0.2653594216072852
1 - coefficient of determination on validation data:
0.3115400506517969
[-31303.71909156  35848.45081517  74506.78099995  -8012.41104949
    671.23713588 100205.53195594  41671.19028923  19507.84532115
 111331.50566184  69959.22677526  23468.73219785 -78236.93092911
   6535.34729956 -28197.21476235  83701.76486765 -21647.26671149
  22056.22833416  -2002.69401407]
536831.9203413767


Is there a way to make a NN network learn a linear model?

Let's first check what the loss used by MLPRegressor...

In [9]:
#let's write the code to learn a linear model with NN: how? Maybe let's use a 
# activation function that is the identity

#let's define the model
mlp_lr = MLPRegressor( activation='identity', hidden_layer_sizes=(1,), solver ='lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
mlp_lr.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:")
print(str(1 - mlp_lr.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:")
print(str(1 - mlp_lr.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_lr.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_lr.intercepts_)

1 - coefficient of determination on training data:
0.26535942166590454
1 - coefficient of determination on validation data:
0.3115390658280597
[array([[  51.55070235],
       [ -59.02846704],
       [-122.96939596],
       [  13.19466306],
       [  -1.10694448],
       [-165.016906  ],
       [ -68.62491747],
       [ -32.12596414],
       [-183.34020234],
       [-114.96558392],
       [ -38.51551396],
       [ 128.83877577],
       [ -10.76237595],
       [  46.43749972],
       [-137.83874569],
       [  35.64844107],
       [ -36.32214078],
       [   3.29740834]]), array([[-607.24280969]])]
[array([-883.44724722]), array([365.29153421])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [10]:
#COMPLETE

## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [11]:
#let's build a NN with 2 nodes in the only hidden layer

#let's define the model
mlp_1h2n = MLPRegressor( hidden_layer_sizes=(2,), solver ='lbfgs', random_state = numero_di_matricola)

#let's learn the model on training data
mlp_1h2n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:")
print(str(1 - mlp_1h2n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:")
print(str(1 - mlp_1h2n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h2n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h2n.intercepts_)

1 - coefficient of determination on training data:
0.18062204778799107
1 - coefficient of determination on validation data:
0.20740441514299124
[array([[  91.02785476,  -33.27409121],
       [ 120.42656695,   39.33968075],
       [  85.92076767,   72.98500911],
       [-271.77222272,   28.29936852],
       [ -30.70531649,   17.82577452],
       [ 197.58128374,   25.8837917 ],
       [  34.91759912,   37.55681963],
       [  96.98060103,   25.68593256],
       [ 312.79882298,  132.75141944],
       [  85.17210062,   68.79226305],
       [  19.30011756,   23.32841572],
       [-217.58314444,  -81.10852994],
       [  -3.45340229,   20.17182737],
       [-300.96434321,  -26.39947236],
       [ 305.19500455,  144.62969649],
       [-463.25858774,  -16.43013601],
       [ 193.67272465,   53.02219071],
       [-241.36587926,  -11.18371124]]), array([[615.21370428],
       [548.87641687]])]
[array([-1049.80550884,   897.57656351]), array([725.96712488])]


Let's build a NN with 5 nodes in the only hidden layer

In [12]:
#let's build a NN with 5 nodes in the only hidden layer

#let's define the model
#mlp_1h5n = MLPRegressor( hidden_layer_sizes=(5,), solver ='lbfgs', random_state = numero_di_matricola)
#increase number of iterations
mlp_1h5n = MLPRegressor( hidden_layer_sizes=(5,), solver ='lbfgs', random_state = numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_1h5n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:")
print(str(1 - mlp_1h5n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:")
print(str(1 - mlp_1h5n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h5n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h5n.intercepts_)

1 - coefficient of determination on training data:
0.1370568967260022
1 - coefficient of determination on validation data:
0.27222424802795997
[array([[ -284.33365847,    96.27803761,    53.01253519,   179.05990055,
          -34.35841817],
       [  997.09777558,   770.43311149,    82.73287893,  -188.72132599,
         -140.10916998],
       [ -363.78663291,   -92.45776898,   214.96195558,  -419.34016422,
          466.48870847],
       [  589.17240777,  -662.11282963,   -87.5390922 ,  -733.76145245,
          -22.69781878],
       [  407.58838438,  -156.12072706,   418.33176082,   241.58611206,
         -180.25723246],
       [ -798.71003436,   315.08517771,   888.88491834,  -561.15843845,
         -268.59285461],
       [ -246.54818872,   336.61560222,  -226.95728971,  -159.93313779,
          200.33740145],
       [ -503.79737536,   566.01810705,  -126.95661736,   566.74381049,
           38.63716691],
       [  -49.69401148,   952.10148572,   874.95044804,   245.93818344,
        

Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [13]:
#let's build a NN with 10 nodes in the only hidden layer

#COMPLETE

Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [14]:
#let's build a NN with 100 nodes in the only hidden layer

#let's define the model
mlp_1h100n = MLPRegressor( hidden_layer_sizes=(100,), solver ='lbfgs', random_state = numero_di_matricola)
#increase number of iterations
#mlp_1h100n = MLPRegressor( hidden_layer_sizes=(100,), solver ='lbfgs', random_state = numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_1h100n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:")
print(str(1 - mlp_1h100n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:")
print(str(1 - mlp_1h100n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_1h100n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_1h100n.intercepts_)

1 - coefficient of determination on training data:
0.03136574832078798
1 - coefficient of determination on validation data:
0.5077609114801309
[array([[  97.54507994,  -22.76091889,   72.43108061, ..., -113.83973374,
         166.58014844,  -87.11530376],
       [   1.65000195,   62.96623943,   87.62914088, ...,   42.99603515,
         -34.82754175,  -23.36184336],
       [ -79.99495266,  -76.89027299,  101.26692363, ...,   29.55581075,
         244.08315418,   44.06499317],
       ...,
       [  51.81643158,  -66.40815001,   68.42703848, ..., -386.33086966,
          62.72326481,  -38.41792385],
       [ -45.89230415,   81.15333297,  151.71563255, ..., -110.51516548,
         -70.8132794 ,  -41.61959806],
       [  11.60944224, -170.00691113,   43.61986964, ...,   67.04285114,
         -14.6294163 ,    4.40164108]]), array([[ 112.47362182],
       [  41.84803871],
       [ -22.71028178],
       [ -43.78426112],
       [ 212.54746239],
       [  60.68904474],
       [  55.03439662],
  

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [15]:
#let's build a NN with 2 hidden layers, 1 node each

#let's define the model
#mlp_2h1n1n = MLPRegressor( hidden_layer_sizes=(1,1,), solver ='lbfgs', random_state = numero_di_matricola)
#increase number of iterations
mlp_2h1n1n = MLPRegressor( hidden_layer_sizes=(1,1,), solver ='lbfgs', random_state = numero_di_matricola, max_iter=2000)

#let's learn the model on training data
mlp_2h1n1n.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("1 - coefficient of determination on training data:")
print(str(1 - mlp_2h1n1n.score(Xtrain_scaled,Ytrain)))

#let's print the error (1 - R^2) on validation data
print("1 - coefficient of determination on validation data:")
print(str(1 - mlp_2h1n1n.score(Xval_scaled,Yval)))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(mlp_2h1n1n.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(mlp_2h1n1n.intercepts_)

1 - coefficient of determination on training data:
0.2371101237960359
1 - coefficient of determination on validation data:
0.27157753824546704
[array([[ -59.38408437],
       [ 125.3697467 ],
       [ 209.90870604],
       [ -79.89485454],
       [  -2.13924742],
       [ 245.46974672],
       [  97.04467526],
       [  75.55082557],
       [ 370.53582766],
       [ 202.16064192],
       [  58.05041681],
       [-248.15927781],
       [  18.50204716],
       [-166.1175824 ],
       [ 351.10050127],
       [-209.42440551],
       [  88.81821826],
       [  -4.34636652]]), array([[0.96994704]]), array([[423.0836986]])]
[array([287.21083913]), array([751.74845424]), array([485.42578143])]


Let's try 2 layers, 2 nodes each

In [16]:
#let's build a NN with 2 layers, 2 nodes each

#COMPLETE

Let's try 2 layers, 10 nodes each

In [17]:
#let's build a NN with 2 layers, 10 nodes each

#COMPLETE

Let's try 2 layers, 100 nodes each

In [18]:
#let's build a NN with 2 layers, 100 nodes each

#COMPLETE

So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [19]:
from sklearn.model_selection import GridSearchCV

mlp_cv = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,21)],
              'activation': ['relu'],
              'solver': ['lbfgs'], 
              'random_state': [numero_di_matricola]
             }
mlp_GS = GridSearchCV(mlp_cv, param_grid=param_grid, 
                   cv=5, verbose=True)
mlp_GS.fit(Xtrain_and_val_scaled, Ytrain_and_val)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                11, 12, 13, 14, 15, 16, 17, 18,
                                                19, 20],
                         'random_state': [1], 'solver': ['lbfgs']},
             verbose=True)

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [20]:
#let's print the best model according to grid search
print("Best model: ",mlp_GS.best_estimator_)
#let's print the error 1-R^2 for the best model
print("Error (1-R^2) of best model: ",1. - mlp_GS.best_score_)

Best model:  MLPRegressor(hidden_layer_sizes=5, random_state=1, solver='lbfgs')
Error (1-R^2) of best model:  0.22155403846328947


Let compare the error of the best NN on train and validation and on test data.

In [21]:
print("Error best model on train and validation: ",1. - mlp_GS.best_estimator_.score(Xtrain_and_val_scaled,Ytrain_and_val))
print("Error best model on test data: ",1. - mlp_GS.best_estimator_.score(Xtest_scaled,Ytest))

Error best model on train and validation:  0.15048994598618193
Error best model on test data:  0.2299589915099295


Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [22]:
#LR the linear regression model
LR = linear_model.LinearRegression()

#fit the model on training data
LR.fit(Xtrain_and_val_scaled, Ytrain_and_val)

print("1 - coefficient of determination on training data:"+str(1 - LR.score(Xtrain_and_val_scaled,Ytrain_and_val)))
print("1 - coefficient of determination on test data:"+str(1 - LR.score(Xtest_scaled,Ytest)))

1 - coefficient of determination on training data:0.2715568577139226
1 - coefficient of determination on test data:0.3373644878767468


Note: MLPRegressor has several other parameters!